In [3]:
import pandas as pd
import numpy as np
import urllib
import json

#import locations of river data
stations=pd.read_csv('river_loc_data_2.csv')


In [4]:
#lists to be filled in loop below
stn_lith=[]
stn_t_age=[]
stn_b_age=[]
map_rank=[]
stn_macro_units=[]
stn_strat_names=[]

#loop through station locations
for i,s in stations.iterrows():
    print('\r%s out of %s points queried' % (i+1,len(stations)),end='')
    
    #macrostrat API for geologic map information at station lat / long
    url='https://macrostrat.org/api/v2/geologic_units/map?lat=%s&lng=%s' % (s['lat'],s['long'])
#     url='https://macrostrat.org/api/v2/geologic_units/map?lat=40.588848&lng=-109.465414'

    #load the data as a list of dictionaries
    tmp=urllib.request.urlopen(url)
    tmp=json.load(tmp)['success']['data']
    
    #ditch any map polygons that do not have lithologies
    tmp=[t for t in tmp if len(t['liths'])!=0]
    
    #dataframe of returned: map sources, map scale and map "rank"
    tmp_ranks=pd.DataFrame(list(zip([t['source_id'] for t in tmp],
                                [sources[t['source_id']] for t in tmp],
                      [source_ranks[sources[t['source_id']]] for t in tmp])),
            columns=['source_id','scale','rank'])

    #record the lowest (largest) map rank found
    map_rank.append(np.min(tmp_ranks['rank']))
    
    #find sources that match this largest scale
    min_ranks=tmp_ranks['source_id'][tmp_ranks['rank']==np.min(tmp_ranks['rank'])].tolist()
    
    #only save map polygons that match this largest scale
    tmp=[t for t in tmp if t['source_id'] in min_ranks]
    
    #gather lithology, age, and unit information for this lat/long
    liths=[]
    t_age=[]
    b_age=[]
    macro_units=[]
    strat_names=[]
    
    #loop through all map polygons
    for t in tmp:
        #lithology IDs
        liths.extend(t['liths'])
        
        #macrostrat units
        macro_units.extend(t['macro_units'])
        
        #strat name_ids
        strat_names.extend(t['strat_names'])
        
        #age information, if it exists
        if t['t_age']!=None:
            t_age.append(t['t_int_age'])
        if t['b_age']!=None:
            b_age.append(t['b_int_age'])
            
    #record lithology and unit information as pipe-separated string
    stn_lith.append('|'.join({str(l) for l in liths}))
    stn_macro_units.append('|'.join({str(l) for l in macro_units}))
    stn_strat_names.append('|'.join({str(l) for l in strat_names}))

    #record top age as AVERAGE of all top ages found
    if len(t_age)!=0:
        stn_t_age.append(np.mean(np.array(t_age)))
    else:
        stn_t_age.append(-9999.0)
        
    #record bottom age as AVERAGE of all bottom ages found
    if len(b_age)!=0:
        stn_b_age.append(np.mean(np.array(b_age)))
    else:
        stn_b_age.append(-9999.0)
        
        
#append all gathered data to original dataframe
stations['lith']=stn_lith
stations['stn_t_age']=stn_t_age
stations['stn_b_age']=stn_b_age
stations['stn_macro_units']=stn_macro_units
stations['stn_strat_names']=stn_strat_names
stations['map_rank']=map_rank
        
#save your work
stations.to_csv('stations_macrostrat.csv',index=False)

1 out of 792 points queried

NameError: name 'sources' is not defined

In [10]:
#load in previously macrostrat-linked stations
stations=pd.read_csv('stations_macrostrat.csv')

#download macrostrat's carbonate lithology library
carb_liths=urllib.request.urlopen('https://macrostrat.org/api/v2/defs/lithologies?lith_type=carbonate')
carb_liths=json.load(carb_liths)['success']['data']

#dolomite lithology IDs
dol_liths={c['lith_id'] for c in carb_liths if c['name']=='dolomite' or c['name']=='dolostone'}
#all carbonate lithology IDs
carb_liths={c['lith_id'] for c in carb_liths}

#download macrostrat's igneous lithology library
igneous_liths=urllib.request.urlopen('https://macrostrat.org/api/v2/defs/lithologies?lith_class=igneous')
igneous_liths=json.load(igneous_liths)['success']['data']
igneous_liths={c['lith_id'] for c in igneous_liths}

#download macrostrat's siliciclastic lithology library
sil_liths=urllib.request.urlopen('https://macrostrat.org/api/v2/defs/lithologies?lith_type=siliciclastic')
sil_liths=json.load(sil_liths)['success']['data']
sil_liths={c['lith_id'] for c in sil_liths}

#lists to be filled in loop below
frac_dol=[]
frac_carb=[]
frac_igneous=[]
frac_sil=[]

#loop through each station
for i,c in stations.iterrows():
    #make lithologies in this station into a set of integers (if lithologies were found)
    if c['lith']!='' and type(c['lith'])!=float:
        tmp_liths=set([int(l) for l in c['lith'].split('|')])
    else:
        tmp_liths={-9999}
    
    #tabulate fraction dolomite lithologies found
    tmp=tmp_liths.intersection(dol_liths)
    frac_dol.append(len(tmp)/len(tmp_liths))
    
    #tabulate fraction carbonate lithologies found
    tmp=tmp_liths.intersection(carb_liths)
    frac_carb.append(len(tmp)/len(tmp_liths))
    
    #tabulate fraction igneous lithologies found
    tmp=tmp_liths.intersection(igneous_liths)
    frac_igneous.append(len(tmp)/len(tmp_liths))

    #tabulate fraction siliciclastic lithologies found
    tmp=tmp_liths.intersection(sil_liths)
    frac_sil.append(len(tmp)/len(tmp_liths))
    
#record these fractions into original dataframe
stations['frac_dol']=frac_dol
stations['frac_carb']=frac_carb
stations['frac_sil']=frac_sil
stations['frac_igneous']=frac_igneous

#save your work
stations.to_csv('stations_macrostrat.csv',index=False)